# Decision-making

This contains three lectures for the module 2:

1. Lecture 2.1 - Real Data sets and coding
2. Lecture 2.2 - Testing and modifying systems
3. Lecture 2.3 - Fairness and Arrow's Impossibility Theorem

## Lecture 2.1 - Real Data sets and coding

We have two major questions to answer:

1. How can we get a computer to automatically compute the winner in each method?
2. How long does it take to compute the winner in each of the methods we have studied?

Answering the first question will help us examine real world data sets, where computing the winner by hand takes too long. I'm sure you've noticed how much time and computation it takes to find the Borda winner of an election. Thankfully for us, computers are good at doing basic arithmetic.

The second question is one that comes up often when dealing with algorithms: What good is an algorithm if it takes years to complete? We will see how we can measure the time it takes to run an algorithm.

### Getting a computer to do the tedious work

Our goal in this section is to write code so that a computer can automotically compute the winner of an election using some of the methods we've studied.

We will focus on getting a spreadsheet, like Excel or Google Sheets, to make these computations, although if you're more comfortable with Python or Scratch you can code it in those.

We will use the following election data that appears on page 4 and 5 of the textbook:

In [ ]:
mathelec = { 
    'ABCD': 14, 
    'CBDA': 10, 
    'DCBA':  8, 
    'BDCA':  4, 
    'CDBA':  1
}

print(mathelec)

In order to get a computer to find the winner of the **plurality method**, we will need to do several steps:

1. Represent the preference schedule in a spreadsheet.
2. Extract the number of first place votes for every candidate.
3. Find the maximum number of first place votes.
4. Display the name of the winner.

**Task 0**. Think about each of these steps and how you might solve them.

Once you've thought about it, check out our partially completed spreadsheet here:

https://docs.google.com/spreadsheets/d/1TM8Nl-jn_QS5UqvlqaobTcvfa4jAe2QZ9dwkEXpM_fA/edit?usp=sharing

Copy the spreadsheet so that you can complete the missing cells. The preference schedule has been input on the left. The data on the right counts the number of first place, second place, third place and fourth place of each candidate. For example, cell `H5` contains a `24`, meaning that candidate B received 24 second place votes.

**Task 1**. Fill in the missing cells (`B11` to `B14`) by pointing to the appropriate cell. For example, if you want cell `B11` to contain the number in cell `G7`, then in cell B11 write `=G7`. We point to cells that way we can change the preference schedule and still automatically generate the winner.

Once you've completed the task, the winner of the plurality method should automatically appear in cell `D11` as candidate A. We've coded this part for you.

Now let's complete the **Borda count method**. To do this we will need to do the following steps:

1. Represent the preference schedule in a spreadsheet. (**Already done!**)
2. Extract the number of first place votes, second place votes, etc. for every candidate. (**Already done!**)
3. Give the appropriate amount of points for the number of first place votes, second place votes, etc. for each candidate.
4. Sum up the points for each candidate to get their Borda score.
5. Find the maximum Borda score.
6. Display the name of the winner.

**Task 2**. Fill in the missing cells (columns B through E, rows 18 to 21) by giving the appropriate amount of points for every first place votes, second place vote, etc. 

**Task 3**. Fill in the missing cells (`F18` to `F21`) by summing the previous four cells. Use the `=SUM()` function.

Once you've completed the task, the winner of the plurality method should automatically appear in cell `H18` as candidate B. We've coded this part for you.

We will leave as an exercise for you coding the **method of pairwise comparisons**.

### Real data sets

Once you've completed your spreadsheet, you can analyze the winners of various elections using various methods. The following contains code (written in `Python`) that computes winners of elections with 3 candidates. It gets a little bit cranky when there are lots of ties, but otherwise implements the methods we've studied.

Play around with the sliders which control the preference schedule. The output at the bottom is:

* The winner of the **plurality method**, and the number of first place votes for the winner.
* The winner of the **Borda count method**, and the Borda score of the winner.
* The winner of the **pairwise comparison method**, and the "pairwise comparison score" of the winner.
* The winner of the **plurality-with-elimination method**, and the number of first place votes for the winner after eliminations.

In [ ]:
from ipywidgets import Button, HBox, VBox, Text, widgets, Layout
from IPython.display import display
from Voting_methods import *

import itertools

items_layout = Layout( width='100px')

people = ['Amrit','Bolade','Carmen']             # Committee candidates.

items = [Button(description=w, layout=items_layout) for w in people]  # Button for each committee member.
perms = list(itertools.permutations(items))      # Every possible permutation of committees.
boxes = [VBox(p) for p in perms]                 # Box for every permutation.

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='50%')

display(HBox(boxes), layout = box_layout)

sliders = [widgets.IntSlider(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Votes',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='d',
    layout=items_layout
) for w in boxes]

# For convenient function input.
letters = ['a','b','c','d','e','f']

# Current function for output.
def f(a,b,c,d,e,f):
    election = {'ABC':a, 'ACB':b, 'BAC':c, 'BCA':d, 'CAB':e, 'CBA':f}
    #print('{} + {} + {} + {} + {} + {} = {}'.format(a, b, c, d, e, f, a+b+c+d+e+f), '\nmike')
    print(plurality_winner(election))
    print(borda_winner(election))
    print(pairwise_comparison_winner(election))
    print(plurality_elimination_winner(election))

out = widgets.interactive_output(f, {letters[i]: sliders[i] for i in range(len(sliders))})

display(HBox(sliders), layout = box_layout)
display(HBox([out]))

While playing around with this tool, make observations about how the winners change as you modify the preference schedule. Is it easy to find situations where the methods produce different winners?

### How long does it take to compute a winner?

One thing you may have noticed is that the different methods take varying amounts of time to actually compute. The plurality method is fairly quick to compute, but the Borda count method involves counting many more things. Our first step is to make this more precise and introduce some helpful notions.

The **complexity of a voting method** is the number of additions added to the number of comparisons that must be performed on a preference schedule to determine the winner. We will express this as a function of the number of candidates, $n$. (If using a variable $n$ is uncomfortable for you, imagine that it is $n=5$ as you go through this analysis.)

Let's examine the **complexity of the Borda count method** as an example. To do this we will copy the steps from the algorithm above, and write down how many steps each takes.

1. Represent the preference schedule in a spreadsheet.
2. Extract votes.
    1. Extract the number of first place votes for every candidate.
    2. Extract the number of second place votes for every candidate.
    3. ...
    4. Extract the number of $n$th place votes for every candidate.
3. Assign points.
    1. Give the appropriate amount of points for the number of first place votes for each candidate.
    2. Give the appropriate amount of points for the number of second place votes for each candidate.
    3. ...
    4. Give the appropriate amount of points for the number of $n$th place votes for each candidate.
4. Sum up the points for each candidate to get their Borda score.
5. Find the maximum Borda score.
6. Display the name of the winner.

When adding up the number of first place votes for each candidate, at most we will have to add up a number for every column in the preference schedule - every different column corresponds to a different ballot. We will calculate the largest number of unique ballots later, but for now we'll denote it by `number_of_ballots(n)`; we definitely expect the number of unique ballots to depend on the number of candidates $n$.

So in step 2, for each candidate we will need to make `number_of_ballots(n)` many additions. Since there are $n$ candidates, in total step 2 will take $n \times$ `number_of_ballots(n)` many additions. 

After we've done this, we have a table that tells us how many first place votes, second place votes, ... each candidate received. *How many entries does this table have?* It has $n$ columns (one for each candidate) and $n$ rows (one for each ballot ranking). To complete step 3 we need to translate each of these entries to their corresponding number of Borda points. For example, each first place vote should award $n$ points. Since we need to make a multiplication for each entry in this table, we will need to make $n \times n$ many multiplications in step 3.

Now we find the Borda score for each candidate by summing up each column. Each column has $n$ entries, so we make $n$ additions for each column. So step 4 has a total of $n \times n$ additions.

In step 5 we find the maximum of $n$ values. We can compute this exactly, but for now let's represent the number of comparisons we need to make as `find_max(n)`. Finally we display the maximum of these Borda counts to get the winner, which does not require any computations or comparisons.

Step | Number of comparisons or additions
----|----
1 | 0
2 | $n \times$ `number_of_ballots(n)`
3 | $n^2$
4 | $n^2 $
5 | `find_max(n)`
6 | 0

The total number of computations we need to do will be the sum of these:

> `total_computations` = 0 + $n \times$ `number_of_ballots(n)` + $n^2$ + $n^2$ + `find_max(n)` + 0  
> = $n \times$ `number_of_ballots(n)` + $ 2 n^2$ + `find_max(n)`

This formula is mostly complete, we only need to express `number_of_ballots(n)` and `find_max(n)` as explicit functions that depend on `n`. We will leave it as an exercise to show that `number_of_ballots(n)` $\leq n \times (n-1) \times (n-2) \times \ldots 3 \times 2 \times 1 = n!$, and `find_max(n)` = $n$.

So:

> `total_computations` = $n \times$ `number_of_ballots(n)` + $ 2 n^2$ + `find_max(n)` = $n \times n! + 2n^2 + n$.

Of these three terms that are being summed, which one is the biggest? Except in very small cases, $n \times n!$ is much, much larger than the other two terms.

$n$ | $n \times n!$ | $2n^2$ | $n$
----|----|----|----
1|1|2|1
2|4|8|2
3|18|18|3
4|96|32|4
5|600|50|5
6|4320|72|6
7|35280|98|7
8|322560|128|8
9|3265920|162|9
10|36288000|200|10
11|439084800|242|11
12|5748019200|288|12
13|80951270400|338|13
14|1220496076800|392|14
15|19615115520000|450|15
16|334764638208000|512|16
17|6046686277632000|578|17
18|115242726703104000|648|18
19|2311256907767808000|722|19

In [ ]:
import math

for n in range(1,20):
    print(str(n) + "|" + str(n * math.factorial(n)) + "|" + str(2*n*n) + "|" + str(n))

Since the $n \times n!$ terms is so much larger than the other terms we can think of the Borda count as taking approximately $n \times n!$ steps.

You might object to use using `number_of_ballots(n)` = $n!$, since you beleive that *usually* there won't actually be that many ballots. In this case it could be more realistic to say:

> The total number of computations needed to find the Borda count winner is approximately $n \times $ `number_of_ballots(n)` when the number of ballots is large (at least $n$), and it is approximately $2n^2$ when the number of ballots is small (less than $n$).

### Exercises

1. In a spreadsheet, or whatever programming language you're comfortable with, code the method of pairwise comparisons.
2. Here you will analyze the number of computations needed for the plurality method.
    1. Show that the plurality method takes at most `number_of_ballots(n)` + `find_max(n)` many computations to find the winner.
    2. Compare this to how many computations the Borda count needs. Which method is faster to compute?
3. Show that if you have a list of 10 numbers, then you can find the maximum number of this list by making only 9 comparisons. More generally, show that if you have a list of $n$ numbers, then you can find the maximum using only $n-1$ comparisons.
4. Show that for $3$ candidates, there can be at most $3 \times 2 \times 1 = 6$ unique ballots. Show that for $4$ candidates there can be at most $4 \times 3 \times 2 \times 1 = 24$ unique ballots.
5. [**Project**] Show that the Borda count method always needs more computation than the plurality-with-elimination method.
6. [**Project**] Investigate how reasonable it is to use the approximation that the number of ballots is $n!$, for $n$ candidates. For example, imagine that the entire population of the earth ranked every person in the world to determine the world's greatest cook. What is the most number of unique ballots that could occur? How close is it to `(Population_of_the_Earth)!` ? What might be a better approximation for the number of unique ballots?
7. [Read about how many computations a computer can do every second.](https://www.popsci.com/intel-teraflop-chip) For the computer in that article, use the table we produced for the Borda count computation to figure out the largest number of candidates this computer could handle in an hour. Assume that the number of unique ballots is $n!$.

## Lecture 2.2 - Testing systems

While investigating voting systems last week we looked at properties systems can have that capture various notions of fairness. In this lecture we will:

* Review the four criteria.
* Review what it means for criteria to be violated.
* How do we find data sets that show a criteria is violated?

### Review of the four fairness criteria

To review, we looked at these four fairness criteria:

1. **The majority criterion**. If candidate X has a majority of the first place votes, then candidate X should be the winner of the election.
2. **The Condorcet criterion**. If candidate X is preferred by the voters over each of the other candidates in a head-to-head comparison, then candidate X should be the winner of the election.
3. **The monotonicity criterion**. If candidate X is a winner of an election and, in a reelection, the only changes in the ballots are changes that favour X (and only X), then X should remain a winner in the election.
4. **The independence-of-irrelevant-alternatives criterion (IIA)**. If candidate X is a winner of an election and in a recount one of the *nonwinning* candidates withdraws or is disqualified, then X should still be a winner of the election.

Voting systems can have some of these and not others. **Exercise**. To help remember what we've covered so far, fill in the following table of information by replacing `False` with `True` in the relevant spots. Use the examples in the textbook to help you. (If you're desperate for the answers, they are contained in the conclusion section of the textbook.)

In [1]:
plurality = {
            'Majority': False,
            'Condorcet': False,
            'Monotone': False,
            'IIA': False
            }
borda_count = {
            'Majority': False,
            'Condorcet': False,
            'Monotone': False,
            'IIA': False
            }
plurality_with_elimination = {
            'Majority': False,
            'Condorcet': False,
            'Monotone': False,
            'IIA': False
            }
pairwise_comparisons = {
            'Majority': False,
            'Condorcet': False,
            'Monotone': False,
            'IIA': False
            }

##########################
# Do not edit below here ######################
##########################
from Voting_methods import *
check_properties(plurality, borda_count, plurality_with_elimination, pairwise_comparisons)

Plurality contains an error. Try again.
Borda Count contains an error. Try again.
Plurality-with-elimination contains an error. Try again.
Pairwise Comparisons contains an error. Try again.


When a criterion is true for a voting system (or it *holds*) it means that every possible election always satisfies that condition. For a criterion to be false (or it *fails*) there only needs to be a single election where the criterion is not satisfied. In mathematical language we say that an election that shows that criterion fails is a **witness** to this criterion failing.

Up until now we have not addressed how to come up with elections that witness that a voting system fails a fairness criteria. It will be a little bit of an art, but we can use computational thinking tools to help us focus our thinking.

### Example - Borda Count fails the majority condition

There are examples of elections where a candidate receives a majority of first place votes, but does not win the election. We will construct a witness of this by using computational thinking. First we list as many relevant facts and observations as we know.

1. We will have candidate A have a majority of first place votes, but candidate B will win the election.
2. Candidate B will win by having more points than A.
    1. How can candidate B have lots of points?
    2. How can candidate A have very few points?
3. A simple election is preferred so that it is easier to make computations and see the underlying behaviour of the election.
    1. We will likely need at least three candidates, and it's probably easier to use more candidates. (*Later on we can remove extra candidates to try to find the simplest election with a property.*)
    2. It's probably easier if there are fewer voters. (*Later on we can remove extra voters to simplify the election.*)

Before we fill in a preference schedule, let's answer questions 2.1 and 2.2. There are many possible answers, but here's one set of answers:

1. Candidate B can have lots of points by having a lot of second place votes.
2. Candidate A can have very few points by having a lot of last place votes.

In keeping with part 3 above, let's try an election with 3 candidates (A,B,C) and let's only use some of the columns of a preference schedule.

|n|m
---|---|---
1st |A|C
2nd |B|B
3rd |C|A

Now we ask if there are choices for n (the number of ABC ballots) and m (the number of CBA ballots) so that n > m (so that A has a majority of the first place ballots) and that the Borda score of B is higher than the other Borda scores. Sadly, this is not possible! (**Exercise**. Compute the Borda scores for this preference shedule as functions of n and m, and show that if n > m then A always has the highest Borda score.)

Partly this means that we were trying to be *too* simple. So at this stage let's fix it by making it slightly less simple. Here are some options:

1. Allow B to have some first place votes when possible, or
2. Introduce another candidate D that always gets 3rd place.

We will try the first option, but the second option is left as an **exercise** for you. Letting B get some first place votes turns our preference schedule to:

|n|m
---|---|---
1st |A|B
2nd |B|C
3rd |C|A

Now at this stage we could write out the Borda scores of each candidate as functions of n and m (with n > m) and then identify the minimal n and m that will give B the highest Borda score. That requires some thinking about inequalities, so instead let's look at small values of n and m and hopefully get lucky! We can always come back to inequalities if we can't find n and m through luck.

Here is a table where the column is the value of n, and the column is the value of m. The entry is the candidate with the highest Borda count, and we include a `T` if there is a tie.

In [ ]:
def score_A(n,m):
    return 3*n + m
def score_B(n, m):
    return 2*n + 3*m

def winner(n,m):
    if m >= n:
        return ' '
    if score_A(n,m) > score_B(n,m):
        return 'A'
    if score_A(n,m) < score_B(n,m):
        return 'B'
    if score_A(n,m) == score_B(n,m):
        return 'T'

def display_table(N):
    data = [[ "|"+winner(n,m) for n in range(1, N)] for m in range(0, N-1)]

    print("  "+"".join(["|"+str(i) for i in range(1,N)]))
    print("---"+"".join(["|---" for i in range(1,N)]))
    for i, row in enumerate(data):
        print(str(i)+ " "+"".join(row))
    return None

display_table(5)

  |1|2|3|4|5|6|7|8|9|10|11|12|13|14|15|16|17|18|19
---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---
0 |A|A|A|A|A|A|A|A|A|A|A|A|A|A|A|A|A|A|A
1 | |T|A|A|A|A|A|A|A|A|A|A|A|A|A|A|A|A|A
2 | | |B|T|A|A|A|A|A|A|A|A|A|A|A|A|A|A|A
3 | | | |B|B|T|A|A|A|A|A|A|A|A|A|A|A|A|A
4 | | | | |B|B|B|T|A|A|A|A|A|A|A|A|A|A|A
5 | | | | | |B|B|B|B|T|A|A|A|A|A|A|A|A|A
6 | | | | | | |B|B|B|B|B|T|A|A|A|A|A|A|A
7 | | | | | | | |B|B|B|B|B|B|T|A|A|A|A|A
8 | | | | | | | | |B|B|B|B|B|B|B|T|A|A|A
9 | | | | | | | | | |B|B|B|B|B|B|B|B|T|A
10 | | | | | | | | | | |B|B|B|B|B|B|B|B|B
11 | | | | | | | | | | | |B|B|B|B|B|B|B|B
12 | | | | | | | | | | | | |B|B|B|B|B|B|B
13 | | | | | | | | | | | | | |B|B|B|B|B|B
14 | | | | | | | | | | | | | | |B|B|B|B|B
15 | | | | | | | | | | | | | | | |B|B|B|B
16 | | | | | | | | | | | | | | | | |B|B|B
17 | | | | | | | | | | | | | | | | | |B|B
18 | | | | | | | | | | | | | | | | | | |B

Using this we see that $n=3$ and $m=2$ produces a Borda winner of B. Let's capture that in a preference schedule:

> This is an election that shows that the Borda count method fails the Majority criterion.

|3|2
---|---|---
1st |A|B
2nd |B|C
3rd |C|A

### Example - Plurality fails the IIA condition

Let's construct another example. This time we will find a witness election for the failure of the IIA condition for the Plurality method.

Again, we list our observations and requirements:

1. We will have candidate A have a plurality of first place votes, but candidate B will have a plurality of first place votes when candidate C drops out.
2. Changing the totals of first place votes:
    1. How can candidate B gain first place votes?
    2. Can candidate A not gain any first place votes?
3. The same simplicity preferences as before.

One answer to question 2.1 is for candidate C to be "blocking" some of the votes of candidate B. That is, there are ballots where C is in first place and B is in second place. That way when C is eliminated, those ballots turn into first place votes for B. This reasoning also helps us with question 2.2, as we can ensure that C is always ranked below A so that no new first place votes will become "unblocked".

This gives us a preference schedule:

|n|m|k
---|---|---|---
1st |A|B|C
2nd |B|C|B
3rd |C|A|A

To ensure that A wins this election we need A to have a plurality of these votes, or in other words:

> n > m, and    
> n > k.

When C drops out of the election the preference schedule becomes:

|n|m|k
---|---|---|---
1st |A|B|~~C~~
2nd |B|~~C~~|B
3rd |~~C~~|A|A

or condensing this into:

|n|m+k
---|---|---
1st |A|B
2nd |B|A

Here B will win if m+k > n. So in total we need three conditions:

> n > m, and    
> n > k, and    
> m + k > n

This is asking for a number n that is individually larger than m and k, but when combined they are larger than n. As long as m and k are close to n this will happen. For example, we can take $n=3$, $m=2$, $k=2$ and get:

|3|2|2
---|---|---|---
1st |A|B|C
2nd |B|C|B
3rd |C|A|A

with winner A, then after C drops out we get:

|3|4
---|---|---
1st |A|B
2nd |B|A

### Exercises

1. Fill in the table of what fairness criteria hold for voting systems, which appears at the beginning of the section.
2. For the Borda count example we used option 1 to find a witness example. Instead, use option 2 to find a witness election.
3. Express in words the special structure of the witness elections that we studied above.
4. Our example election for how the plurality method fails IIA also shows that plurality-with-elimination fails IIA. 
    1. Verify this.
    2. Does this example also show that the Borda Count method fails IIA? Either verify this, or find an election that witnesses that the Borda Count method fails IIA.

## 2.3 - Arrow's Impossibility Theorem

Start by reading the Conclusion section of Chapter 1 in the textbook.

### Arrow's impossibility theorem (dictator version)

The version of Arrow's Impossibility theorem presented in the textbook is this one:

> **Arrow's Impossibility Theorem** (Textbook version). It is mathematically impossible for a voting system to satisfy all four of the fairness criteria we discussed: the Majority criteria, the Condorcet Criterion, the Monotonicity criterion, and IIA.

There is another version of this theorem which is more commonly called "Arrow's impossibility theorem". Since it's the more common one, we'll introduce it here, but we won't spend too much time with it. To introduce it we need two new notions.

> **The Unanimity Criterion**. If all voters prefer candidate A to candidate B, then the voting system ranks candidate A above candidate B.

> **Dictator**. A dictator is a person whose ballot is always the outcome of the election.

Now we can state the classic version of Arrow's Impossibility Theorem. One major difference is that it concerns voting systems that *ranks* all the candidates, not just ones that merely produce a winner.

> **Arrow's Impossibility Theorem** (Dictator version). Every voting system that ranks all the candidates that satisfies IIA and the Unanimity criterion must contain a dictator.

### How do rules interact with each other?

One interesting thing about these theorems is that they are able to say something about all possible voting systems at once, not just about specific systems. This theorem is really about how *properties* or *fairness criteria* interact and relate to each other. While this may feel strange at first, we have some familiarity with how properties interact. For example, in the world of numbers we know that the property "n is even" is incompatible with "n is odd", but the property "n is a multiple of 6" guarantees that "n is even".

The relationships are more complicated in Arrow's theorem, but we will show an example of how properties can interact and how to establish those facts. These arguments are different from showing that a particular voting system (like the Borda Count method) does or does not have a property. We argue that one property is related to another one, *for every system*. In exercise 1 below you will see this.

### Exercises

1. Show that if a system satisfies the Condorcet criterion, then it must also satisfy the Majority criterion. (Hint: Show that if a candidate has the majority of first place votes, then they must also be the Condorcet candidate.)
2. Give an example of a voting system that satisfies the Majority criterion, but not the Condorcet criterion.
3. Discuss the validility of the following interpretation of Arrow's Impossibility Theorem: "The only fair decision-making system is a dictatorship".